In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dlib

In [2]:
for i in range(100):
    globals()['land'+str(i)] = pd.read_csv('./dlib(denorm)/train/%s.csv'%i)
    globals()['land'+str(i)] = np.array(globals()['land'+str(i)])
    
    tr_x = globals()['land'+str(i)][:, 0]
    tr_y = globals()['land'+str(i)][:, 1]
    
    tr_cx = ((min(tr_x)+max(tr_x)) / 2)        #dlib_rect 중점
    tr_cy = ((min(tr_y)+max(tr_y)) / 2)
    
    tr_h = max(tr_y)-min(tr_y)    #종축 길이
    
    tr_x -= tr_cx
    tr_y -= tr_cy
    
    norm_trx = (tr_x - min(tr_x)) / tr_h
    norm_try = (tr_y - min(tr_y)) / (tr_h *1.5)
    
    globals()['train_norm'+str(i)] = np.column_stack((norm_trx, norm_try))

In [3]:
## eye, nose, mouth, jaws
for i in  range(100):
    globals()['norm_eye'+str(i)] = globals()['train_norm'+str(i)][36:48].flatten()
    globals()['norm_nose'+str(i)] = globals()['train_norm'+str(i)][27:36].flatten()
    globals()['norm_mouth'+str(i)] = globals()['train_norm'+str(i)][48:].flatten()
    globals()['norm_jaws'+str(i)] = globals()['train_norm'+str(i)][:17].flatten()

In [89]:
# for i in range(100):
#     globals()['mark'+str(i)] = pd.read_csv('./dlib(denorm)/test/%s.csv'%i)
#     globals()['mark'+str(i)] = np.array(globals()['mark'+str(i)])
    
#     te_x = globals()['mark'+str(i)][:, 0]
#     te_y = globals()['mark'+str(i)][:, 1]
    
#     te_cx = ((min(te_x)+max(te_x)) / 2)
#     te_cy = ((min(te_y)+max(te_y)) / 2)

#     te_h = max(te_y)-min(te_y)
    
    
#     te_x -= te_cx
#     te_y -= te_cy

#     norm_tex = (te_x - min(te_x)) / te_h
#     norm_tey = (te_y - min(te_y)) / (te_h *1.5)
    
#     globals()['test_norm'+str(i)] = np.column_stack((norm_tex, norm_tey))

In [90]:
# ## eye, nose, mouth, jaws
# for i in  range(100):
#     globals()['norm_eye2'+str(i)] = globals()['test_norm'+str(i)][36:48].flatten()
#     globals()['norm_nose2'+str(i)] = globals()['test_norm'+str(i)][27:36].flatten()
#     globals()['norm_mouth2'+str(i)] = globals()['test_norm'+str(i)][48:].flatten()
#     globals()['norm_jaws2'+str(i)] = globals()['test_norm'+str(i)][:17].flatten()

In [12]:
face_path = './pixelsize/800.png'
face_origin = cv2.imread(face_path)
face_copy = face_origin.copy()
face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
detector = dlib.get_frontal_face_detector()
dlib_rects = detector(face_rgb, 1)
    
img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
img = []
    
for dlib_rect in dlib_rects:
    points = predictor(face_rgb, dlib_rect)
    list_points = list(map(lambda p: [p.x, p.y], points.parts()))
    img.append(list_points)
    img = np.array(img)
    
land = np.zeros((68,2))
    
for j in range(68):
    land[j][0] = img[0][j][0]
    land[j][1] = img[0][j][1]
        
x = land[:,0]
y = land[:,1]
    
x -= ((min(x) + max(x)) / 2)
y -= ((min(y) + max(y)) / 2)
    
norm_x = (x - min(x)) / (max(y)-min(y))
norm_y = (y - min(y)) / ((max(y)-min(y)) * 1.5)
    
norm = np.column_stack((norm_x, norm_y))
norm_eye = norm[36:48].flatten()
norm_nose = norm[27:36].flatten()
norm_mouth = norm[48:].flatten()
norm_jaws = norm[:17].flatten()

In [13]:
try_eye = pd.read_csv('./dataset/train/train.csv')
try_eye = try_eye.iloc[:, 16:22]
try_nose = pd.read_csv('./dataset/train/train.csv')
try_nose = try_nose.iloc[:, 22:30]
try_mouth = pd.read_csv('./dataset/train/train.csv')
try_mouth = try_mouth.iloc[:, 30:]
try_jaws = pd.read_csv('./dataset/train/train.csv')
try_jaws = try_jaws.iloc[:, 5:9]

In [14]:
tey_eye = pd.read_csv('./dataset/test/test.csv')
tey_eye = tey_eye.iloc[:, 16:22]
tey_nose = pd.read_csv('./dataset/test/test.csv')
tey_nose = tey_nose.iloc[:, 22:30]
tey_mouth = pd.read_csv('./dataset/test/test.csv')
tey_mouth = tey_mouth.iloc[:, 30:]
tey_jaws = pd.read_csv('./dataset/test/test.csv')
tey_jaws = tey_jaws.iloc[:, 5:9]

In [15]:
train_x = np.zeros((100, len(norm_jaws0)))   #morm0 -> norm_eye, nose, mouth, jaws
for i in range(100):
    for j in range(len(norm_jaws0)):
        train_x[i][j] = globals()['norm_jaws'+str(i)][j]   

In [184]:
# test_x = np.zeros((100, len(norm_nose0)))   #morm0 -> norm_eye, nose, mouth, jaws
# for i in range(100):
#     for j in range(len(norm_nose0)):
#         test_x[i][j] = globals()['norm_nose'+str(i)][j]   

In [16]:
test_x = np.zeros((1, len(norm_jaws)))
for j in range(len(norm_jaws)):
    test_x[0][j] = norm_jaws[j]

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [17]:
model = LinearRegression()
model.fit(train_x,try_jaws)
y_pred = model.predict(test_x)
prediction = pd.DataFrame(y_pred)

In [11]:
prediction   #500px

,0,1,2,3
0,0.383744,0.316083,0.453818,0.559744


In [18]:
prediction    #800px

,0,1,2,3
0,0.566343,0.347744,0.581708,0.519096


In [178]:
prediction   #eye

,0,1,2,3,4,5
0,0.371326,0.631646,0.607577,0.487716,0.437961,0.538062


In [173]:
prediction   #nose

,0,1,2,3,4,5,6,7
0,0.378432,0.415773,0.493757,0.380154,0.320162,0.522676,0.521505,0.636121


In [182]:
prediction   #mouth

,0,1,2,3
0,0.190955,0.505558,0.478015,0.562302


In [187]:
prediction #jaws

,0,1,2,3
0,0.303808,0.534138,0.384855,0.339651


In [70]:
# tey_eye

In [108]:
mae = mean_absolute_error(tey_nose, y_pred)
mae

0.10756064228202006